In [90]:
! cd ../../genepeeks-science/ && python setup.py develop

/home/ec2-user/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg/setuptools/dist.py:331: UserWarning: Normalizing '1.0.1.1-dev' to '1.0.1.1.dev0'
running develop
running egg_info
writing requirements to genepeeks.egg-info/requires.txt
writing genepeeks.egg-info/PKG-INFO
writing top-level names to genepeeks.egg-info/top_level.txt
writing dependency_links to genepeeks.egg-info/dependency_links.txt
reading manifest file 'genepeeks.egg-info/SOURCES.txt'
writing manifest file 'genepeeks.egg-info/SOURCES.txt'
running build_ext
Creating /home/ec2-user/anaconda2/lib/python2.7/site-packages/genepeeks.egg-link (link to .)
genepeeks 1.0.1.1.dev0 is already the active version in easy-install.pth

Installed /home/ec2-user/git/genepeeks-science
Processing dependencies for genepeeks==1.0.1.1.dev0
Searching for apiclient==1.0.3
Best match: apiclient 1.0.3
Adding apiclient 1.0.3 to easy-install.pth file

Using /home/ec2-user/anaconda2/lib/python2.7/site-packages
Searching for config==0.

In [4]:
from genepeeks.common import utilities as util
import pysam, os
import tabletext
import numpy as np

#### Get DMD exons from gene collection in mongo using genepeeks science utilities function

In [5]:
DMD_ensembl = util.Mongo.get_collection_data('gene', wanted_db='prod', query={'_id': 'DMD'}, find_one=True, single_field='ensembl')
DMD_exons = util.get_nested_value(DMD_ensembl, ('is_primary', 'transcripts', 'is_primary', 'exons'))
DMD_exons_merged = util.merge_intervals(DMD_exons, min_dist=200, include_index=True)
exon_labels = ['Ex' + exon['index'] for exon in DMD_exons_merged]
print exon_labels


Finished loading ensembl within the gene collection from the genepeeks-prod database with a single entries with a query of {'_id': 'DMD'} after 0.17 sec
['Ex1', 'Ex2', 'Ex3', 'Ex4', 'Ex5', 'Ex6', 'Ex7', 'Ex8', 'Ex9', 'Ex10', 'Ex11', 'Ex12', 'Ex13', 'Ex14', 'Ex15', 'Ex16', 'Ex17', 'Ex18', 'Ex19', 'Ex20', 'Ex21', 'Ex22', 'Ex23', 'Ex24', 'Ex25', 'Ex26', 'Ex27', 'Ex28', 'Ex29', 'Ex30', 'Ex31', 'Ex32', 'Ex33', 'Ex34', 'Ex35', 'Ex36', 'Ex37', 'Ex38', 'Ex39', 'Ex40', 'Ex41', 'Ex42', 'Ex43', 'Ex44', 'Ex45', 'Ex46', 'Ex47', 'Ex48', 'Ex49', 'Ex50', 'Ex51', 'Ex52', 'Ex53', 'Ex54', 'Ex55', 'Ex56', 'Ex57', 'Ex58', 'Ex59', 'Ex60', 'Ex61', 'Ex62', 'Ex63', 'Ex64', 'Ex65|66', 'Ex67', 'Ex68', 'Ex69', 'Ex70', 'Ex71', 'Ex72', 'Ex73', 'Ex74', 'Ex75', 'Ex76', 'Ex77', 'Ex78', 'Ex79']


#### Get each sample's coverage across all DMD exons

In [49]:
base_headers = ['subject', 'specimen', 'sample', 'gender', 'sequencer', 'flow_cell_id', 'lane']
coverage_matrix = [base_headers + exon_labels]
# print coverage_matrix
# exit()
for root, dirs, files in os.walk('/mnt/vep/subjects'):
    for file_name in files:
        if file_name.endswith('.bam'):# and file_name.startswith('F'):
            # Initialize all exons to coverage of 0 for each subject
#             subject_coverage = [0] * len(DMD_exons)
#             subject = os.path.splitext(file_name)[0]
            file_path = os.path.join(root, file_name)
            subject_coverages = {}
            bamfile = pysam.AlignmentFile(file_path, "rb")
            for RG in bamfile.header['RG']:
                subject, specimen_sample, flow_cell_lane = RG['ID'].split('_')
                gender = subject[0]
                specimen_num, sequencer, sample = specimen_sample.split('-')
                specimen = '{}_{}'.format(subject, specimen_num)
                sample = '{}_{}'.format(subject, specimen_sample)
                flow_cell_id, lane = flow_cell_lane.rsplit('-', 1)
                row = [subject, specimen, sample, gender, sequencer, flow_cell_id, lane]
                subject_coverages[RG['ID']] = row + [0] * len(DMD_exons_merged)

            for read in bamfile.fetch('X', start=31137345, end=33229636):
                if not read.is_unmapped and read.mapping_quality == 60:
                    # Find what exon each read falls in, and increase that exon's coverage by 1
                    interval_info = util.in_interval(read.reference_start, DMD_exons_merged, get_interval=True)
                    if not interval_info[0]:
                        # If the start of the read is not in an exon, check the end of the read
                        interval_info = util.in_interval(read.reference_end, DMD_exons_merged, get_interval=True)
                    # else:
                    #     # Make sure that the end of the read does not map to a different exon than the start
                    #     end_interval_info = util.in_interval(read.reference_end, DMD_exons_merged, get_interval=True)
                    #     if end_interval_info[0] and end_interval_info[1] != interval_info[1]:
                    #         print read.reference_start, interval_info[1], read.reference_end, end_interval_info[1]
                        
                    if interval_info[0]:
                        exon_num = interval_info[1]
                        subject_coverages[read.get_tag('RG')][exon_num + len(base_headers)] += 1
                        
            coverage_matrix += subject_coverages.values()
coverage_matrix = np.array(coverage_matrix)
print coverage_matrix

[['subject' 'specimen' 'sample' ..., 'Ex77' 'Ex78' 'Ex79']
 ['MRMR-0011-266B' 'MRMR-0011-266B_1' 'MRMR-0011-266B_1-M1-1' ..., '70'
  '13' '58']
 ['MRMR-0011-266B' 'MRMR-0011-266B_1' 'MRMR-0011-266B_1-M1-1' ..., '53'
  '16' '55']
 ..., 
 ['FRMR-00SD-679G' 'FRMR-00SD-679G_1' 'FRMR-00SD-679G_1-M1-1' ..., '58' '8'
  '47']
 ['FRMR-00SD-679G' 'FRMR-00SD-679G_1' 'FRMR-00SD-679G_1-M1-1' ..., '73' '6'
  '51']
 ['FRMR-00SD-679G' 'FRMR-00SD-679G_1' 'FRMR-00SD-679G_1-M1-1' ..., '66'
  '14' '69']]


#### Print each exon's mean, std, min, and max coverage, and turn the stats into an array

In [50]:
print len(coverage_matrix), 'rows'
exon_cov_stats = [['Exon', 'Mean', 'Std', 'Min', 'Max']]
exon_max_cov = []
for i in range(len(DMD_exons_merged)):
    col = i + len(base_headers)
    exon_coverage = map(int, coverage_matrix[1:,col])
    exon_cov_stats.append([coverage_matrix[0][col], round(np.mean(exon_coverage), 2), round(np.std(exon_coverage), 2), min(exon_coverage), max(exon_coverage)])
# exon_stats_array = np.array(exon_cov_stats[1:,[1:5]])
print tabletext.to_text(exon_cov_stats)

145 rows
┌─────────┬────────┬────────┬─────┬─────┐
│ Exon    │ Mean   │ Std    │ Min │ Max │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex1     │ 319.42 │ 112.96 │ 121 │ 649 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex2     │  12.02 │    6.3 │   1 │  33 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex3     │  76.68 │  28.02 │  27 │ 142 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex4     │ 129.99 │  51.16 │  44 │ 260 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex5     │  75.72 │  27.66 │  22 │ 143 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex6     │ 114.24 │  43.24 │  27 │ 228 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex7     │ 114.05 │  40.92 │  48 │ 212 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex8     │  82.72 │  30.86 │  30 │ 149 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex9     │  25.42 │  10.69 │   6 │  50 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex10    │ 107.52 │  43.47 │  25 │ 206 │
├─────────┼────────┼────────┼─────┼─────┤
│ Ex11    │  79.59 │  30.

#### Normalize each subject's coverage on each exon compared to the mean and std of that exon's coverage

In [15]:
coverage_matrix_vs_max_exon = coverage_matrix / exon_stats_array[:,-1]
print coverage_matrix_vs_max_exon

normalized_coverage_matrix = (coverage_matrix - exon_stats_array[:,1]) / exon_stats_array[:,2]
print normalized_coverage_matrix

[[ 0.67197452  0.34883721  0.56981132 ...,  0.7337884   0.79245283
   0.73668639]
 [ 0.65059145  0.41860465  0.55471698 ...,  0.6894198   0.49056604
   0.73964497]
 [ 0.78207461  0.93023256  0.68301887 ...,  0.83959044  0.73584906
   0.74556213]
 ..., 
 [ 0.58826206  0.62790698  0.57735849 ...,  0.47098976  0.73584906
   0.5443787 ]
 [ 0.51319381  0.37209302  0.35471698 ...,  0.49829352  0.58490566
   0.53846154]
 [ 0.39126479  0.44186047  0.40754717 ...,  0.56313993  0.43396226
   0.46153846]]
[[-0.09158981 -1.14850299 -0.52688771 ..., -0.18404207  0.81959379
   0.12059012]
 [-0.24263264 -0.78922156 -0.63833937 ..., -0.48125286 -1.09199522
   0.14197135]
 [ 0.68612013  1.84550898  0.30899972 ...,  0.52469136  0.46117085
   0.1847338 ]
 ..., 
 [-0.68290645  0.28862275 -0.47116188 ..., -1.94444444  0.46117085
  -1.26918965]
 [-1.21316322 -1.02874251 -2.11507384 ..., -1.7615455  -0.49462366
  -1.31195211]
 [-2.07442877 -0.66946108 -1.72499303 ..., -1.32716049 -1.45041816
  -1.86786402]]


#### Get a matrix of the probability that any subject's reads will be for any particular exon

In [51]:
probability_in_an_exon_matrix = [list(coverage_matrix[0])]
# print coverage_matrix.shape[1]
# print coverage_matrix[1:, [(len(base_headers)+1):coverage_matrix.shape[1]-1]]
# total_subject_reads = np.sum(map(int, coverage_matrix[1:,[len(base_headers)+1:-1]]), axis=1)
for row in coverage_matrix[1:]:
    total_subj_reads = sum(map(int, row[len(base_headers):]))
    new_row = list(row[:len(base_headers)])
    new_row += [float(item) / total_subj_reads for item in row[len(base_headers):]]
    probability_in_an_exon_matrix.append(new_row)
probability_in_an_exon_matrix = np.array(probability_in_an_exon_matrix)
print probability_in_an_exon_matrix

[['subject' 'specimen' 'sample' ..., 'Ex77' 'Ex78' 'Ex79']
 ['MRMR-0011-266B' 'MRMR-0011-266B_1' 'MRMR-0011-266B_1-M1-1' ...,
  '0.0136213271064' '0.00252967503405' '0.0112862424596']
 ['MRMR-0011-266B' 'MRMR-0011-266B_1' 'MRMR-0011-266B_1-M1-1' ...,
  '0.0104805220486' '0.0031639311845' '0.0108760134467']
 ..., 
 ['FRMR-00SD-679G' 'FRMR-00SD-679G_1' 'FRMR-00SD-679G_1-M1-1' ...,
  '0.0109186746988' '0.00150602409639' '0.00884789156627']
 ['FRMR-00SD-679G' 'FRMR-00SD-679G_1' 'FRMR-00SD-679G_1-M1-1' ...,
  '0.0129570465034' '0.0010649627263' '0.00905218317359']
 ['FRMR-00SD-679G' 'FRMR-00SD-679G_1' 'FRMR-00SD-679G_1-M1-1' ...,
  '0.012497633024' '0.00265101306571' '0.0130657072524']]


#### Get summary stats for each subject across all exons

In [16]:
subject_normalized_stats = [['Subject', 'Mean', 'Min', 'Max', 'Norm-Mean', 'Norm-Min', 'Norm-Max']]
for i, subject in enumerate(subjects):
    subject_stats = [subject, round(np.mean(coverage_matrix_vs_max_exon[i]), 3), round(min(coverage_matrix_vs_max_exon[i]), 3), round(max(coverage_matrix_vs_max_exon[i]), 3)]
    subject_stats += [round(np.mean(normalized_coverage_matrix[i]), 3), round(min(normalized_coverage_matrix[i]), 3), round(max(normalized_coverage_matrix[i]), 3)]
    subject_normalized_stats.append(subject_stats)
print tabletext.to_text(subject_normalized_stats)

┌────────────────┬───────┬───────┬───────┬───────────┬──────────┬──────────┐
│ Subject        │ Mean  │ Min   │ Max   │ Norm-Mean │ Norm-Min │ Norm-Max │
├────────────────┼───────┼───────┼───────┼───────────┼──────────┼──────────┤
│ FRMR-00SC-714V │ 0.677 │ 0.349 │ 0.986 │     -0.27 │   -1.378 │     1.44 │
├────────────────┼───────┼───────┼───────┼───────────┼──────────┼──────────┤
│ FRMR-00CS-821F │ 0.711 │ 0.419 │ 0.917 │     -0.04 │   -1.471 │    1.261 │
├────────────────┼───────┼───────┼───────┼───────────┼──────────┼──────────┤
│ FRMR-0081-1987 │ 0.818 │ 0.534 │   1.0 │     0.683 │   -0.485 │    1.846 │
├────────────────┼───────┼───────┼───────┼───────────┼──────────┼──────────┤
│ FRMR-0085-579V │  0.66 │ 0.381 │ 0.856 │    -0.404 │   -1.817 │    0.765 │
├────────────────┼───────┼───────┼───────┼───────────┼──────────┼──────────┤
│ FRMR-00DM-853C │ 0.606 │ 0.302 │ 0.782 │    -0.762 │   -2.083 │    0.175 │
├────────────────┼───────┼───────┼───────┼───────────┼──────────┼──────────┤

In [52]:
! git status

On branch master
Your branch is up-to-date with 'origin/master'.
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   coverage_matrix.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../MAPQ_distribution.ipynb
	modified:   coverage_matrix.ipynb



In [53]:
! git add coverage_matrix.ipynb

In [54]:
! git commit -m "Adding sample info to the coverage matrix"

[master 2948311] Adding sample info to the coverage matrix
 1 file changed, 942 insertions(+), 463 deletions(-)
 rewrite notebooks/coverage_matrix.ipynb (67%)


In [ ]:
! git push origin

Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple' mode was introduced in Git 1.7.11. Use the similar mode
'current' instead of 'simple' if you sometimes use older versions of Git)

Username for 'https://github.com': 